In [1]:
# import models
import mplfinance as mpf
from config import api_key
import requests
import pandas as pd
import time
import datetime
from datetime import time
import matplotlib.pyplot as plt
import cufflinks as cf
import os


In [6]:
#input info used to pass through api 
symbol = input("What ticker do you want to search?")
periodtype = input("Period type (day, month, year)")
frequency = input("Frequency of data")
frequencytype = input("Frequency type (minute, daily)")
period = input("Period of data")
needExtendedHoursData = input('Extended hours data (True or False)')


What ticker do you want to search?MSFT
Period type (day, month, year)day
Frequency of data15
Frequency type (minute, daily)minute
Period of data10
Extended hours data (True or False)False


In [16]:
# input the start and end timeframe of the data imported as milliseconds in epoch
end = input("end")
start = input("start")

end1645370298000
start1644506298000


In [17]:
#url for td
url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)
    
#parameters
key = {'apikey':api_key,
         'periodType':periodtype,
          'frequencyType': frequencytype,
          'frequency': frequency,
          'period': period,
          'needExtendedHoursData': needExtendedHoursData,
          'endDate': end,
          'startDate': start
         }
#requesting data
content = requests.get(url = url, params=key)

data = content.json()
data

{'candles': [{'open': 303.99,
   'high': 305.37,
   'low': 303.33,
   'close': 304.63,
   'volume': 2419058,
   'datetime': 1644503400000},
  {'open': 304.64,
   'high': 307.75,
   'low': 304.64,
   'close': 307.35,
   'volume': 1366457,
   'datetime': 1644504300000},
  {'open': 307.45,
   'high': 309.12,
   'low': 306.95,
   'close': 307.68,
   'volume': 1370567,
   'datetime': 1644505200000},
  {'open': 307.67,
   'high': 307.81,
   'low': 306.2,
   'close': 306.4111,
   'volume': 1214977,
   'datetime': 1644506100000},
  {'open': 306.405,
   'high': 308.2752,
   'low': 305.83,
   'close': 307.94,
   'volume': 1368945,
   'datetime': 1644507000000},
  {'open': 307.92,
   'high': 308.61,
   'low': 306.0,
   'close': 306.21,
   'volume': 1117049,
   'datetime': 1644507900000},
  {'open': 306.22,
   'high': 306.65,
   'low': 304.69,
   'close': 304.8,
   'volume': 1182403,
   'datetime': 1644508800000},
  {'open': 304.79,
   'high': 305.42,
   'low': 304.29,
   'close': 305.28,
   'volu

In [18]:
#create arrays for each category
open_tick = []
high_tick = []
low_tick = []
close_tick = []
volume_tick = []
datetime_tick = []
#iterrates through data to append each array with it's value
for i in range(len(data["candles"])):
    open_tick.append(data["candles"][i]["open"])
    high_tick.append(data["candles"][i]["high"])
    low_tick.append(data["candles"][i]["low"])
    close_tick.append(data["candles"][i]["close"])
    volume_tick.append(data["candles"][i]["volume"])
    datetime_tick.append(data["candles"][i]["datetime"])

#converting epoch to datetime       
file = []
for i in range(len(datetime_tick)):
    # divide epoch by 1000 because it is in milliseconds
    test = datetime_tick[i]/1000
    file.append(datetime.datetime.fromtimestamp(test))

#convert to dataframe
theta_df = pd.DataFrame({
            "Open":open_tick,
            "Close":close_tick,
            "High":high_tick,
            "Low":low_tick,
            "Volume":volume_tick,
            "Datetime": file
    })

In [19]:
# set index to Datetime
theta_df = theta_df.set_index("Datetime")

In [20]:
# Calculate % gain for the day 
theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

theta_df

,Open,Close,High,Low,Volume,Gain%
Datetime,,,,,,
2022-02-10 06:30:00,303.990,304.6300,305.3700,303.330,2419058,0.210533
2022-02-10 06:45:00,304.640,307.3500,307.7500,304.640,1366457,0.889575
2022-02-10 07:00:00,307.450,307.6800,309.1200,306.950,1370567,0.074809
2022-02-10 07:15:00,307.670,306.4111,307.8100,306.200,1214977,-0.409172
2022-02-10 07:30:00,306.405,307.9400,308.2752,305.830,1368945,0.500971
...,...,...,...,...,...,...
2022-02-18 11:45:00,289.330,289.9300,290.2200,288.765,495619,0.207376
2022-02-18 12:00:00,289.890,289.1950,290.5277,288.960,582571,-0.239746
2022-02-18 12:15:00,289.180,288.2900,289.4200,287.860,756214,-0.307767


In [21]:
# for the stock ticker if a folder does not exist already, one will be created
mypath = f"{symbol}"
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [23]:
# uploads csv to that folder specifying the frequency and type to identify the csv
upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
theta_df.to_csv(upload_file, index = False)